In [ ]:
import requests
from bs4 import BeautifulSoup
import pdfkit
from PyPDF2 import PdfMerger
import os
import asyncio
from playwright.async_api import async_playwright

# путь к wkhtmltopdf.exe
path_wkhtmltopdf = r"C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe"
config = pdfkit.configuration(wkhtmltopdf=path_wkhtmltopdf)

OUTPUT_DIR = "books"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def get_chapter_links(start_url):
    """Парсим боковое меню и получаем список ссылок на все главы"""
    resp = requests.get(start_url)
    soup = BeautifulSoup(resp.text, "html.parser")

    links = []
    for a in soup.select("a"):
        href = a.get("href")
        if href and (href.startswith("/docs/Machine-learning/") or href.startswith("/docs/Neural-networks/")):
            full_url = "https://deepmachinelearning.ru" + href
            if full_url not in links:
                links.append(full_url)
    return links


def save_chapter_pdf(url, filename):
    """Сохраняем одну главу в PDF"""
    options = {
        "enable-local-file-access": None,  # обязательно для картинок/MathJax
        "javascript-delay": "2000",       # подождать 2 сек, пока прогрузится MathJax
        "no-stop-slow-scripts": None,
        "quiet": ""
    }
    pdfkit.from_url(url, filename, configuration=config, options=options)
    print(f"Сохранено: {filename}")
    return filename


def build_book(start_url, book_name):
    """Собираем весь учебник в единый PDF"""
    links = get_chapter_links(start_url)
    print(f"[{book_name}] Найдено {len(links)} глав")

    merger = PdfMerger()
    for i, link in enumerate(links, 1):
        chapter_file = os.path.join(OUTPUT_DIR, f"{book_name}_chapter_{i:02d}.pdf")
        save_chapter_pdf(link, chapter_file)
        merger.append(chapter_file)

    final_file = os.path.join(OUTPUT_DIR, f"{book_name}.pdf")
    merger.write(final_file)
    merger.close()
    print(f"Книга сохранена: {final_file}")


if __name__ == "__main__":
    build_book("https://deepmachinelearning.ru/docs/Machine-learning/book-title", "machine_learning")
    build_book("https://deepmachinelearning.ru/docs/Neural-networks/book-title", "deep_learning")


[machine_learning] Найдено 20 глав
Сохранено: books\machine_learning_chapter_01.pdf
Сохранено: books\machine_learning_chapter_02.pdf
Сохранено: books\machine_learning_chapter_03.pdf
Сохранено: books\machine_learning_chapter_04.pdf
Сохранено: books\machine_learning_chapter_05.pdf
Сохранено: books\machine_learning_chapter_06.pdf
Сохранено: books\machine_learning_chapter_07.pdf
Сохранено: books\machine_learning_chapter_08.pdf
Сохранено: books\machine_learning_chapter_09.pdf
Сохранено: books\machine_learning_chapter_10.pdf
Сохранено: books\machine_learning_chapter_11.pdf
Сохранено: books\machine_learning_chapter_12.pdf
Сохранено: books\machine_learning_chapter_13.pdf
Сохранено: books\machine_learning_chapter_14.pdf
Сохранено: books\machine_learning_chapter_15.pdf
Сохранено: books\machine_learning_chapter_16.pdf
Сохранено: books\machine_learning_chapter_17.pdf
Сохранено: books\machine_learning_chapter_18.pdf
Сохранено: books\machine_learning_chapter_19.pdf
Сохранено: books\machine_learning_

In [ ]:
import asyncio
from playwright.async_api import async_playwright

async def save_book(url, filename):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")
        await page.pdf(path=filename, format="A4", print_background=True)
        await browser.close()
        print(f"Сохранено: {filename}")

if __name__ == "__main__":
    asyncio.run(save_book(
        "https://deepmachinelearning.ru/docs/Machine-learning/book-title",
        "machine_learning.pdf"
    ))


Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Connection.run() done, defined at c:\Users\liali\database_redact\.venv\lib\site-packages\playwright\_impl\_connection.py:303> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\liali\AppData\Local\Programs\Python\Python310\lib\asyncio\tasks.py", line 232, in __step
    result = coro.send(None)
  File "c:\Users\liali\database_redact\.venv\lib\site-packages\playwright\_impl\_connection.py", line 310, in run
    await self._transport.connect()
  File "c:\Users\liali\database_redact\.venv\lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "c:\Users\liali\database_redact\.venv\lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    self._proc = await asyncio.create_subprocess_exec(
  File "C:\Users\liali\AppData\Local\Programs\Python\Python310\lib\asyncio\subprocess.py", line 218, in create_subprocess_exec
    transport

NotImplementedError: 